<a href="https://colab.research.google.com/github/Babitdor/News_SummarizerT5Transformer/blob/main/News_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename

In [3]:
from datasets import load_dataset
import pandas as pd
import torch
import re
import evaluate
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import (
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    AutoModelForSeq2SeqLM,
)
import nltk

nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
!nvidia-smi

Fri Nov 15 11:44:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Using only a subset of the CNN/Daily Dataset
subset_size = 5000
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0", split='train').shuffle(seed=42).select(range(subset_size))
full_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)
train_dataset = full_dataset['train']
val_dataset = full_dataset['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# data_save_path = "Enter Desired PATH"

# train_dataset.to_csv(f'{data_save_path}/train_df.csv', sep="\t", encoding="utf-8", index=False)
# val_dataset.to_csv(f'{data_save_path}/val_df.csv', sep="\t", encoding="utf-8", index=False)

In [6]:
print(train_dataset)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 4000
})


In [7]:
print(val_dataset)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 1000
})


In [9]:
OUTPUT_PATH = "../models"
MODEL = "google-t5/t5-base"
EPOCHS = 1
BATCH_SIZE = 2
EVAL_STEPS = 200
WARM_UP_STEPS = 200
MAX_LENGTH = 512

In [10]:
class Text_Cleaning:

    def __init__(
        self,
        clean_text=False,
        remove_stopwords=False,
        use_stemming=False,
        use_lemmatization=False,
    ):
        self.stop_words = set(stopwords.words("english")) if remove_stopwords else None
        self.stemmer = PorterStemmer() if use_stemming else None
        self.lemmatizer = WordNetLemmatizer() if use_lemmatization else None

    def clean_text(self, text):
        text = re.sub(r"\s+", " ", text)
        text = re.sub(r"[^a-zA-Z\s.,!?\'\";:()]", "", text)
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        return text.lower().strip()

    def remove_stop_words(self, tokens):
        if self.stop_words:
            tokens = [tokens for token in tokens if token not in self.stop_words]
        return tokens

    def stem_tokens(self, tokens):
        if self.stemmer:
            tokens = [self.stemmer.stem(token) for token in tokens]
        return tokens

    def lemmatize_tokens(self, tokens):
        if self.lemmatizer:
            tokens = [
                self.lemmatizer.lemmatize(token)
                for token in tokens
                if isinstance(token, str)
            ]
        return tokens

    def clean_text(self, text):
        tokens = text.split()
        if self.clean_text:
            tokens = self.clean_text(tokens)
        if self.stop_words:
            tokens = self.remove_stop_words(tokens)
        if self.stemmer:
            tokens = self.stem_tokens(tokens)
        if self.lemmatizer:
            tokens = self.lemmatize_tokens(tokens)

        processed_text = " ".join(tokens)

        return processed_text

In [11]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

def preprocess_text(dataset):
    inputs = [f"summarize: {article}" for article in dataset['article']]
    model_inputs = tokenizer(inputs,max_length=MAX_LENGTH, padding="max_length",truncation=True,return_tensors="pt",)

    targets = [summary for summary in dataset['highlights']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets,max_length=MAX_LENGTH, padding="max_length", truncation=True,return_tensors="pt",
            )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# In case dataset requires cleaning

# cleaning = Text_Cleaning(
#     clean_text=True,
#     remove_stopwords=True,
#     use_stemming=True,
#     use_lemmatization=True,
# )

In [12]:
tokenized_train = train_dataset.map(
    preprocess_text,
    batched=True,
    num_proc=4
)
tokenized_valid = val_dataset.map(
    preprocess_text,
    batched=True,
    num_proc=4
)
# tokenized_val_data = preprocess.preprocess_text(val_dataset_dataset)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [23]:
from torch import cuda

class ModelTrainer:
    def __init__(self, train_dataset, val_dataset, model_name="t5-base",
                 preprocess_logits_for_metrics = None, compute_metrics = None):
        self.model_name = model_name
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        device = "cuda" if cuda.is_available() else "cpu"
        self.model.to(device)
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.preprocess_logits_for_metrics = preprocess_logits_for_metrics
        self.compute_metrics = compute_metrics

    def train(
        self,
        output_dir="../models/results",
        epochs=3,
        batch_size=4,
        eval_steps=500,
        learning_rate=2e-4,
        weight_decay=0.01,
        warmup_steps=500,
        fp16=True,
        resume_from_checkpoint=None,
    ):

        training_args = TrainingArguments(
            output_dir=output_dir,
            eval_strategy="steps",
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=epochs,
            eval_steps=eval_steps,
            logging_dir="../models/logs",
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            warmup_steps=warmup_steps,
            save_steps=eval_steps,
            gradient_accumulation_steps=2,
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            preprocess_logits_for_metrics=self.preprocess_logits_for_metrics,
            compute_metrics=self.compute_metrics,
        )

        print("Starting training...")
        trainer.train(resume_from_checkpoint=resume_from_checkpoint)
        print("Training complete!")

    def evaluate(self):
        trainer = Trainer(model=self.model, eval_dataset=self.val_dataset)
        print("Starting evaluation")
        eval_results = trainer.evaluate()
        print("Evaluation complete")
        return eval_results

    def generate_summary(
        self, text, max_length=150, num_beams=4, repetition_penalty=2.0
    ):
        device = next(self.model.parameters()).device

        inputs = tokenizer.encode(
            "summarize: " + text,
            return_tensors='pt',
            max_length=512,
            truncation=True,
            padding='max_length').to(device)

        with torch.no_grad():
            summary_ids = self.model.generate(
                inputs,
                max_length=max_length,
                min_length=50,
                temperature=0.7,
                top_k=50,
                top_p=0.9,
                do_sample=True,
                num_beams=num_beams,
                early_stopping=True,
                repetition_penalty=repetition_penalty,
            )
        decoded_text = tokenizer.decode(
            summary_ids[0], skip_special_tokens=True
        )
        return decoded_text

In [14]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [24]:
trainer = ModelTrainer(
    tokenized_train,
    tokenized_valid,
    model_name=MODEL,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train(
    output_dir=OUTPUT_PATH,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    eval_steps=EVAL_STEPS,
    warmup_steps=WARM_UP_STEPS,
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting training...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,No log,0.214724,0.625500,0.364300,0.586500,73.466000
400,No log,0.217768,0.620300,0.360400,0.580300,73.472000
600,1.053100,0.213199,0.625600,0.366000,0.586200,73.468000
800,1.053100,0.211047,0.629800,0.369000,0.590200,73.473000


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,No log,0.214724,0.625500,0.364300,0.586500,73.466000
400,No log,0.217768,0.620300,0.360400,0.580300,73.472000
600,1.053100,0.213199,0.625600,0.366000,0.586200,73.468000
800,1.053100,0.211047,0.629800,0.369000,0.590200,73.473000
1000,0.249100,0.209719,0.631300,0.372700,0.592800,73.473000


Training complete!


In [ ]:
# Checkpoint Resume Training
#
# trainer.train(resume_from_checkpoint="../models/['Enter checkpoint folder name']")

In [18]:
test_dataset = (
    load_dataset("abisee/cnn_dailymail", "3.0.0", split="test")
    .shuffle(seed=42)
    .select(range(100))
)

In [29]:
import random


# Print the results for comparison for 1 random example

i = random.randint(0, 4)

article = test_dataset[i]["article"]
highlights = test_dataset[i]["highlights"]  # Ground truth summary

# Generate the summary using your model's function

summary = trainer.generate_summary(
    article, max_length=80, num_beams=5, repetition_penalty=1.0
)

print("Original Article:", article)


print("\nGround Truth Summary (Highlights):", highlights)


print("\nGenerated Summary:", summary)

Original Article: British taekwondo fighter Aaron Cook has confirmed he plans to compete for Moldova at the 2016 Olympics in Rio. Dorset-born Cook, 24, who was overlooked for the Great Britain taekwondo squad at London 2012, applied for citizenship after receiving funding from Moldovan billionaire Igor Iuzefovici and has now received his passport from the small eastern European state. 'I will soon begin a new journey representing the Republic of Moldova at all International competitions and hopefully the Rio Olympic games and beyond, should I qualify,' Cook wrote on his Facebook page. Three time European champion Aaron Cook (right) has refused to fight for Great Britain  since May 2012 . The British taekwondo star has been granted citizenship by Moldova and plans to fight for them in Rio 2016 . 'Although I am upset and disappointed I will not represent my country of birth at another major championships, I felt I had no other option. 'I am a fighter at heart and I am not going to throw 

In [20]:
import gradio as gr

In [27]:
iface = gr.Interface(
    fn=trainer.generate_summary,
    inputs=[
        gr.Textbox(lines=7, placeholder="Enter your text to Summarize"),
        gr.Slider(minimum=50, maximum=150, value=1, step=1, label="Max Length"),
        gr.Slider(minimum=1, maximum=5, value=1, step=1, label="Beam"),
    ],
    outputs="text",
    title="Text Summarizer with T5",
    description="This This is a text summarization model using T5. Enter some text, and it will provide a summary.",
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://873f910e4a7cfa72e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
